In [14]:
import tkinter as tk
from tkinter import filedialog, Label, Frame, Canvas, Scrollbar, Button
from zipfile import ZipFile
from PIL import Image, ImageTk
import io
import pandas as pd
import datetime
import pdb
from openpyxl import Workbook
from openpyxl.drawing.image import Image as oimage
import openpyxl
import requests
import os
import zipfile
from datetime import datetime

# 현재 날짜와 시간을 가져옵니다
now = datetime.now()

# 현재 날짜를 "년-월-일" 형식의 문자열로 변환합니다
current_date = now.strftime("%Y%m%d")
current_sec = now.strftime("%f")


In [15]:
script_dir = '/Users/jonathanoh/Desktop/script/crawling'
download_path = '/Users/jonathanoh/Desktop/down'
api_key = '3e027a878a60255d4913a0239e276306'
period = "Day"

In [16]:
# def combine_csv_files_remove_duplicates(directory_path):
#     combined_df = pd.DataFrame()

#     # List to store individual dataframes
#     dfs = []

#     # Iterate through each file in the directory
#     for file in os.listdir(directory_path):
        
#         if file.endswith('.csv'):
#             file_path = os.path.join(directory_path, file)
#             # Read each CSV file
#             df = pd.read_csv(file_path)
#             while len(df.columns)>2:
#                 del df[df.columns[-1]]
#             # Append the dataframe to the list
#             dfs.append(df)
            

#     # Combine all dataframes in the list
#     combined_df = pd.concat(dfs, ignore_index=True)
#     combined_df.columns =['url', 'filename']
#     for file in os.listdir(directory_path):
#         if file.endswith('.csv'):
#             os.remove(os.path.join(download_path, file))
    
#     combined_df.to_csv(f"Civitai_url_data.csv", index=False)

#     # Remove duplicates
#     combined_df.drop_duplicates(inplace=True)
    
# #     for file in os.listdir(directory_path):
# #         if file.endswith('.csv'):
            
#     return combined_df

def combine_csv_files_remove_duplicates(directory_path):
    combined_df = pd.DataFrame()

    # List to store individual dataframes
    dfs = []

    # Iterate through each file in the directory
    for file in os.listdir(directory_path):
        
        if file.endswith('.csv'):
            file_path = os.path.join(directory_path, file)
            # Read each CSV file
            df = pd.read_csv(file_path)
            while len(df.columns)>2:
                del df[df.columns[-1]]
            # Append the dataframe to the list
            dfs.append(df)
            
#     print(dfs)
    # Combine all dataframes in the list
    combined_df = pd.concat(dfs, ignore_index=True)
    if dfs:
        combined_df.columns=['url', 'filename']
    
    for file in os.listdir(directory_path):
        if file.endswith('.csv'):
            os.remove(os.path.join(directory_path, file))
    csv_file = os.path.join(directory_path, "url_data.csv")

    
    combined_df.to_csv(csv_file, index=False)
    
    # Remove duplicates
    combined_df.drop_duplicates(inplace=True)
    
#     for file in os.listdir(directory_path):
#         if file.endswith('.csv'):
            
    return combined_df


def download_image(url, file_path):
    response = requests.get(url, stream=True)
    with open(file_path, 'wb') as file:
        for chunk in response.iter_content(1024):
            file.write(chunk)
            
            
# Civitai API endpoint for images
api_url = "https://civitai.com/api/v1/images"


# Define parameters for popular images
# The actual parameters depend on the API documentation
params = {
    "sort": "Most Reactions",  # 예: 인기도에 따라 정렬
    "order": "desc",       # 내림차순 정렬 (가장 인기 있는 이미지부터)
    "limit": 200,    # 가져올 이미지 수
    "period": period
}

# Make the API request
response = requests.get(api_url, params=params)

# Check if the request was successful
if response.ok:
    # Parse the JSON response
    images_data = response.json()
#     print(images_data)

    # Extract image URLs or other relevant data
    image_urls = [image['url'] for image in images_data['items']] 
#     if 'data' in images_data else []
    
    # Print or process the image URLs
#     for url in image_urls:
#         print(url)
else:
    print("Failed to retrieve images. Status code:", response.status_code)

    
    

duplicate_url = combine_csv_files_remove_duplicates(script_dir)
old_urls = duplicate_url.loc[:, 'url'].to_list()

checkable_data = []


i=1
for item in images_data['items']:    
    if 'url' in item:
        image_url = item['url']
        if not image_url in  old_urls :     
            file_name = f"civitai_{period}_{item['id']}{os.path.splitext(image_url)[1]}"
            checkable_data.append([image_url, file_name])
            file_path = os.path.join(download_path, file_name)
            download_image(image_url, file_path)
            print(f"{i}___{file_name} 다운로드 완료.")
            i=i+1
        else :
            print(f"중복:{image_url}")
if checkable_data :
    download_tracking = pd.DataFrame(checkable_data)
    download_tracking.columns = ['url', 'filename']
    civitai_csv = os.path.join(script_dir, f"Civitai_{period}_{current_date}_{current_sec}.csv")
    download_tracking.to_csv(civitai_csv, index=False)

print('finish')


1___civitai_Day_4905555.jpeg 다운로드 완료.
2___civitai_Day_4898088.jpeg 다운로드 완료.
3___civitai_Day_4905742.jpeg 다운로드 완료.
4___civitai_Day_4905554.jpeg 다운로드 완료.
5___civitai_Day_4903948.jpeg 다운로드 완료.
6___civitai_Day_4898129.jpeg 다운로드 완료.
7___civitai_Day_4905553.jpeg 다운로드 완료.
중복:https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/5aec1b5c-c839-46b9-a2ee-af9b110b18c1/width=1280/5aec1b5c-c839-46b9-a2ee-af9b110b18c1.jpeg
중복:https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/438a3d9f-1627-4e06-b9e3-6ac6851b4474/width=640/438a3d9f-1627-4e06-b9e3-6ac6851b4474.jpeg
8___civitai_Day_4906078.jpeg 다운로드 완료.
중복:https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/f273df36-4752-4668-8d18-343f7bebe9eb/width=1408/f273df36-4752-4668-8d18-343f7bebe9eb.jpeg
중복:https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/d236d903-7ed6-4646-9643-caab81e09dbc/width=1344/d236d903-7ed6-4646-9643-caab81e09dbc.jpeg
중복:https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/5d113218-7e3b-4be5-b946-efce8dabeb4c/width=1440/5d113218-7e3b-4be5-b946-efce8

124___civitai_Day_4904412.jpeg 다운로드 완료.
125___civitai_Day_4890162.jpeg 다운로드 완료.
126___civitai_Day_4903997.jpeg 다운로드 완료.
127___civitai_Day_4904950.jpeg 다운로드 완료.
128___civitai_Day_4905305.jpeg 다운로드 완료.
129___civitai_Day_4888989.jpeg 다운로드 완료.
130___civitai_Day_4894655.jpeg 다운로드 완료.
131___civitai_Day_4897436.jpeg 다운로드 완료.
132___civitai_Day_4904723.jpeg 다운로드 완료.
133___civitai_Day_4907126.jpeg 다운로드 완료.
134___civitai_Day_4907512.jpeg 다운로드 완료.
135___civitai_Day_4907625.jpeg 다운로드 완료.
136___civitai_Day_4907776.jpeg 다운로드 완료.
137___civitai_Day_4908054.jpeg 다운로드 완료.
138___civitai_Day_4908943.jpeg 다운로드 완료.
중복:https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/3b00293b-4f66-4498-bbe2-bf48774a6903/width=1408/3b00293b-4f66-4498-bbe2-bf48774a6903.jpeg
139___civitai_Day_4895263.jpeg 다운로드 완료.
140___civitai_Day_4904603.jpeg 다운로드 완료.
141___civitai_Day_4904641.jpeg 다운로드 완료.
142___civitai_Day_4904677.jpeg 다운로드 완료.
143___civitai_Day_4894620.jpeg 다운로드 완료.
144___civitai_Day_4888532.jpeg 다운로드 완료.
145___civitai_Day_

In [17]:
# filelist = []
for file in os.listdir('.'):
    if file.lower().endswith('.jpeg'):
        file_path= os.path.join(download_path, file)
#         filelist.append(file)
        try :
            image = Image.open(file_path)
        except: 
#             print(file_path)
            correct_extension = ".mov"
            new_file_path = file_path.rsplit('.', 1)[0] + correct_extension
            os.rename(file_path, new_file_path)
#         file_path= os.path.join(download_path, file)
#         correct_file_extension(file_path)

In [18]:
import imghdr

def correct_image_extension(file_path):
    actual_type = imghdr.what(file_path)
    if actual_type:
        new_file_path = file_path.rsplit('.', 1)[0] + '.' + actual_type
        os.rename(file_path, new_file_path)
        return new_file_path
    return file_path

# 예제 사용

# 파일 경로
# download_path = '/Users/jonathanoh/Desktop/down'
# file_list = 

# # 파일 확장자 추론
# file_extension = get_file_extension(file_path)

# # 파일 이름과 확장자 변경
# new_file_path = os.path.splitext(file_path)[0] + '.' + file_extension
# os.rename(file_path, new_file_path)



In [19]:
filelist=[]
for file in os.listdir('.'):
    if file.lower().endswith('.jpeg'):
        filelist.append(file)
        file_path= os.path.join(download_path, file)
        file_extension = correct_image_extension(file_path)
        
        

import mimetypes

# 파일 내용으로부터 MIME 타입 추정
def guess_extension_from_content(file_path):
    mime_type, _ = mimetypes.guess_type(file_path)
    if mime_type:
        extension = mimetypes.guess_extension(mime_type)
        return extension
    return None

# 파일 확장자 수정
def correct_file_extension(file_path):
    correct_extension = guess_extension_from_content(file_path)
    if correct_extension:
        new_file_path = file_path.rsplit('.', 1)[0] + correct_extension
        os.rename(file_path, new_file_path)
        return new_file_path
    return file_path




filelist=[]
for file in os.listdir('.'):
    if file.lower().endswith('.jpg'):
        filelist.append(file)
        file_path= os.path.join(download_path, file)
        correct_file_extension(file_path)
        

In [20]:
def zip_and_delete_images(folder_path, zip_filename):
    # ZIP 파일 경로 설정
    zip_filepath = os.path.join(folder_path, zip_filename + '.zip')

    # 폴더 내의 모든 파일들을 순회하며 이미지 파일만 압축
    with zipfile.ZipFile(zip_filepath, 'w') as zipf:
        for foldername, subfolders, filenames in os.walk(folder_path):
            for filename in filenames:
                # 이미지 파일 확장자 확인
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    file_path = os.path.join(foldername, filename)
                    zipf.write(file_path, filename)

    # 압축 후 원본 이미지 파일 삭제
    for foldername, subfolders, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                file_path = os.path.join(foldername, filename)
                os.remove(file_path)

    print(f"Images in '{folder_path}' have been zipped into '{zip_filepath}' and deleted.")

# 사용 예시

In [21]:


zipfile_name = f"civitai_{period}_{current_date}_{current_sec}"
zip_and_delete_images(download_path, zipfile_name)

Images in '/Users/jonathanoh/Desktop/down' have been zipped into '/Users/jonathanoh/Desktop/down/civitai_Day_20231227_318904.zip' and deleted.


In [22]:
def decode_metadata(value):
    # Handle decoding of metadata if necessary (e.g., bytes to string)
    if isinstance(value, bytes):
        # Try to decode as UTF-8 after removing null bytes
        try:
            cleaned_value = value.replace(b'\x00', b'')
#             print(cleaned_value)
            return cleaned_value.decode('utf-8')
        except UnicodeDecodeError:
            # If UTF-8 decoding fails, try other common encodings
            encodings = ['ascii', 'windows-1250', 'windows-1252', 'iso-8859-1', 'iso-8859-15']
            for encoding in encodings:
                try:
                    return cleaned_value.decode(encoding)
                except UnicodeDecodeError:
                    continue
            # If all decodings fail, represent the binary data as hex
            return cleaned_value.hex()
    elif isinstance(value, str):
        return value
    else:
        return str(value) 

def preprocess_text(text):
    # Replace newlines with spaces and remove excess whitespace
    return ' '.join(text.split())

def extract_j2(text):
    text = preprocess_text(text)
    unicode_index = text.find("UNICODE")
    parameters_index = text.find("parameters:")
    
    # Determine the starting index based on which keyword appears first
    if unicode_index != -1 or parameters_index != -1:
        if unicode_index == -1 or (parameters_index != -1 and parameters_index < unicode_index):
            start_index = parameters_index + len("parameters:")
        else:
            start_index = unicode_index + len("UNICODE")

        end_index_negative = text.find("Negative prompt:", start_index)
        end_index_steps = text.find("Steps:", start_index)
        end_index = min(end_index_negative if end_index_negative != -1 else float('inf'),
                        end_index_steps if end_index_steps != -1 else float('inf'))
        if end_index != float('inf'):
            return text[start_index:end_index].strip()
    return ""



def extract_k2(text):
    text = preprocess_text(text)
    negative_index = text.find("Negative prompt:")
    if negative_index != -1:
        start_index = negative_index + 17  # Adjusted index for "Negative prompt:"
        end_index = text.find("Steps:", start_index)
        if end_index != -1:
            return text[start_index:end_index].strip()
    return ""

def extract_m2(text):
    text = preprocess_text(text)
    steps_index = text.find("Steps:")
    if steps_index != -1:
        # Start the substring from the index of "steps:" to include it
        return text[steps_index:].strip()

def image_data_loading():
    
    img_data= []
    for file in os.listdir('.'):
        if file.lower().endswith('.zip') and file.lower().startswith('civitai'):
            print(f'processing zip file : {file}')
            try:
                zipfile = ZipFile(file, 'r')
                for filename in zipfile.namelist():
    #                 print(image)
                    image_data = zipfile.read(filename)
                    image = Image.open(io.BytesIO(image_data))
                    try:
                        image.thumbnail((180, 180))
                        image.save(filename)
                    except:
                        print(f"썸네일 에러 : {filename}")
#                     print(filename)
#                     thumbnail.save(f"thumbnail_{filename}")

    #                 image = Image.open(image)

                    img_data.append({"image":image, 'metadata': image.info, 'filename': filename})

            except Exception as e:
                print('error')
    return  img_data


def exif_extract(data):
    
    for image_data in data :
        final = []
        for i in range(len(data)):
            temp=[]
            temp.append(data[i]["filename"])
            try :
                temp.append(decode_metadata(data[i]["metadata"]["exif"]))
                temp.append(extract_j2(decode_metadata(data[i]["metadata"]["exif"])))
                temp.append(extract_k2(decode_metadata(data[i]["metadata"]["exif"])))
                temp.append(extract_m2(decode_metadata(data[i]["metadata"]["exif"])))
            except:
                temp.append("no data")
            final.append(temp)
    return final

def image_in_excel(filepath):
    wb = openpyxl.load_workbook(filepath)
    ws = wb.active
    a=[]
    # b=[]
    
    for i in range(ws.max_row-1):
        
        row_number=i+1
        column_letter = "F"
        cell_address = f'{column_letter}{row_number}'
        img_file = f'{ws.cell(row=i+1, column = 1).value}'
#         print(img_file)
        # b.append(img_file)
        if img_file in os.listdir('.'):
            img = oimage(img_file)
            ws.add_image(img, cell_address)
        

    wb.save(filepath)
    delete_image_files_in_current_directory()
    # for x in b:
    #     os.remove(x)
    
def delete_image_files_in_current_directory():
    # 현재 작업 디렉토리를 가져옵니다.
    current_directory = os.getcwd()

    # 현재 작업 디렉토리 내의 모든 파일 목록을 가져옵니다.
    files = os.listdir(current_directory)

    # 이미지 파일 확장자 리스트를 지정합니다. 여기에 원하는 이미지 확장자를 추가하세요.
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']

    # 현재 작업 디렉토리 내의 파일을 순회하며 이미지 파일인 경우 삭제합니다.
    for file in files:
        file_path = os.path.join(current_directory, file)
        # 파일이 존재하고 확장자가 이미지 확장자 중 하나인 경우 삭제합니다.
        if os.path.isfile(file_path) and any(file.lower().endswith(ext) for ext in image_extensions):
            os.remove(file_path)
            print(f"Deleted: {file_path}")

In [23]:

data = image_data_loading()
final = exif_extract(data)
df = pd.DataFrame(final)
df.drop_duplicates(inplace=True)
df.columns = ['filename', 'metadata', 'positive','negative','settings']

# 엑셀 파일로 저장

filename = zipfile_name.rsplit('.', 1)[0] + '.' + "xlsx"

df.to_excel(filename, index=False)

filepath = os.path.join(download_path, filename)
image_in_excel(filepath)

processing zip file : civitai_Day_20231227_318904.zip
processing zip file : civitai_Day_20231225_070258.zip
processing zip file : civitai_Day_20231226_317666.zip
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4876669.jpeg
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4861456.jpeg
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4897781.jpeg
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4856637.jpeg
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4907673.jpeg
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4853209.jpeg
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4853136.jpeg
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4875853.png
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4863646.jpeg
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4878909.jpeg
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4903763.jpeg
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4863187.jpeg
Deleted: /Users/jonathanoh/Desktop/down/civitai_Day_4857821

In [186]:
df = combine_csv_files_remove_duplicates(".")

In [188]:
df.head()

,url,filename
0,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Year_127497.jpeg
1,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Year_175719.jpeg
2,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Year_295008.jpeg
3,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Year_295538.jpeg
4,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Year_212334.jpeg
